<a href="https://colab.research.google.com/github/PARTHIBAN-007/SLM-From-Scratch/blob/main/SLM_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets tiktoken

In [2]:
from datasets import load_dataset

ds = load_dataset("roneneldan/TinyStories")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

(…)-00000-of-00004-2d5a1467fff1081b.parquet:   0%|          | 0.00/249M [00:00<?, ?B/s]

(…)-00001-of-00004-5852b56a2bd28fd9.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00002-of-00004-a26307300439e943.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

(…)-00003-of-00004-d243063613e5a057.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00000-of-00001-869c898b519ad725.parquet:   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

In [ ]:
import tiktoken
import os
import numpy as np
from tqdm.auto import tqdm

enc = tiktoken.get_encoding("gpt2")

def process(example):
  ids = enc.encode_ordinary(example['text'])
  out = {'ids':ids,'len':len(ids)}
  return out


if not os.path.exists('train.bin'):
  tokenized = ds.map(
      process,
      remove_columns = ['text'],
      desc = "tokenizing the splits",
      num_proc = 8
  )

  for split,dset in tokenized.items():
    arr_len = np.sum(dset['len'],dtype = np.uint64)
    filename = f'{split}.bin'
    dtype = np.uint16
    arr = np.memmap(filename,dtype = dtype,mode = "w+",shape = (arr_len,))
    total_batches = 1024

    idx = 0
    for batch_idx in tqdm(range(total_batches),desc = f'writing {filename}'):
      batch = dset.shard(num_shards = total_batches,index = batch_idx , contiguous = True).with_format('numpy')
      arr_batch=  np.concatenate(batch['ids'])
      arr[idx:idx+len(arr_batch)] = arr_batch
      idx += len(arr_batch)
    arr.flush()


tokenizing the splits (num_proc=8):   0%|          | 0/2119719 [00:00<?, ? examples/s]

tokenizing the splits (num_proc=8):   0%|          | 0/21990 [00:00<?, ? examples/s]

writing train.bin:   0%|          | 0/1024 [00:00<?, ?it/s]

In [ ]:
def get_batch(split):
  if split == "train":
    data = np.memmap('train.bin',dtype = np.uint16,mode = 'r')
  else:
    data = np.memmap('validation.bin',dtype=np.uint16,mode = 'r')
  ix = torch.randint(len(data)-block_size,(batch_size,))
  x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
  y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])

  if device_type == "cuda":
    x,y = x.pin_memory().to(device,non_blocking = True), y.pin_memory().to(device,non_blocking = True)
  else:
    x,y = x.to(device) , y.to(device)
  return x.y


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from dataclasses import dataclass
import numpy as np
from tqdm.auto import tqdm
from contextlib import nullcontext
import os

class LayerNorm(nn.Module):
  def __init__(self,ndim,bias):
    self.weight = nn.Parameter(torch.ones(ndim))
    self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None
  def forward(self,x):
    return F.layer_norm(x,self.weight.shape , self.weight,self.bias , 1e-5)


class CausalSelfAttention(nn.Module):
  def __init__(self,config):
    super.__init__()
    assert config.n_embed % config.n_head == 0
    self.c_attn = nn.Linear(config.n_embed,3*config.n_embed,bias= config.bias)
    self.c_proj = nn.Linear(config.n_embed,config.n_embed,bias = config.bias)
    self.nattn_dropout = nn.Dropout(config.dropout)
    self.resid_dropout = nn.Dropout(config.dropout)
    self.n_head = config.n_head
    self.n_embed = config.n_embed
    self.flash = hasattr(F,'scaled_dot_product_attention')
    if not self.flash:
      self.regiser_buffer("bias",torch.tril(torch.ones(config.block_size,config.block_size))
                                  .view(1,1,config.block_size,config.block_size))

  def forward(self,x):
    B , T, C = x.size()
    q,k,v = self.c_attn(x).split(self.n_embed,dim=2)
    k = k.view(B ,T ,self.n_head, C//self.n_head).transpose(1,2)
    q = q.view(B ,T ,self.n_head, C//self.n_head).transpose(1,2)
    v = v.view(B ,T ,self.n_head, C//self.n_head).transpose(1,2)


    if self.flash:
      y  = F.scaled_dot_product_attention(q,k,v,attn_mask= None,dropout_p = self.attn_dropout.p id self.training else 0.0 else is_causal = True)
    else:
      attn = (q@k.transpose(-2,-1))*(1.0/math.sqrt(k.size(-1)))
      att = att.masked_fill(self.bias[:,:,:T,:T]==0 ,float('inf'))
      att = F.softmax(att,dim=-1)
      att = self.attn_droput(att)
      y = att@v

    y = y.transpose(1,2).contiguous().view(B,T,C)
    y = self.resid_dropout(self.c_proj(y))
    return y

class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embed, 4 * config.n_embed, bias=config.bias)
        self.gelu = nn.GELU()
        self.c_proj = nn.Linear(4 * config.n_embed, config.n_embed, bias=config.bias)
        self.dropout = nn.Dropout(config.dropout)
    def forward(self, x):
        return self.dropout(self.c_proj(self.gelu(self.c_fc(x))))

class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln1 = LayerNorm(config.n_embd, config.bias)
        self.attn = CausalSelfAttention(config)
        self.ln2 = LayerNorm(config.n_embd, config.bias)
        self.mlp = MLP(config)
    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x


@dataclass
class GPTConfig:
    block_size: int
    vocab_size: int
    n_layer: int
    n_head: int
    n_embd: int
    dropout: float = 0.0
    bias: bool = True

class GPT(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.config = config
    self.transformer = nn.ModuleDict(dict(
        wte = nn.Embedding(config.vocab_size,config.n_embed),
        wpe = nn.Embedding(config.block_size,config.n_embed),
        drop = nn.Dropout(config.dropout),
        h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
        ln_f = LayerNorm(config.n_embed,config.bias),
    ))
    self.lm_head = nn.Linear(config.n_embed,config.vocab_size,bias = False)
    self.transformer.wte.weight = self.lm_head.weight

    self.apply(self._init_weights):
    for pn,p in self.named_parameters():
      if pn.endswith("c_proj.weight"):
        nn.init.normal_(p,mean =0.0,std =0.02/math.sqrt(2*config.n_layer))

    def _init_weights(self,module):
      if isinstance(module,nn.Linear):
        nn.init.normal_(module.weight,mean=0.0,std =0.02)
        if module.bias is not None:
          nn.init.zeros_(module.bias)
        elif isinstance(module,nn.Embedding):
          nn.init.normal_(module.weight,mean = 0.0,std = 0.02)

    def forward(self,idx,targets = None):
      device = idx.device
      b, t = idx.size()
      assert t<= self.config.block_size
      pos = torch.arange(0,t,dtype = torch.long,device = device)

      tok_emb = self.transformerr.wte(idx)
      pos_emb = self.transformer.wpe(pos)
      x = self.transformer.drop(tok_emb + pos_emb)
      for block in self.transformer.h:
        x = block(x)
      x = self.transformer.ln_h(x)

      if targets is not None:
        logits = self.lm_head(x)
        loss = F.cross_entropy(logits.view(-1,logits.size(-1)),targets.view(-1),ignore_index = -1)
        return logits,loss
      else:
        return self.lm_head(x)


    @torch.no_grad()
    def generate(self,idx,max_new_tokens,temperature=1.0,top_k = None):
      for _ in range(max_new_tokens):
        idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:,-self.config.block_size:]
        logits = self(idx_cond)
        logits = logits[:,-1,:]
        if top_k is not None:
          v  , = torch.topk(logits,min(top_k,logits.size(-1)))
          logits[logits<v[:,[-1]]] = - float('inf')
        probs = F.softmax(logits,dim=-1)
        idx_next = torch.multinomial(probs,num_samples=1)
        idx = torch.cat((idx,idx_next),dim=1)
      return idx

In [ ]:
config = GPTConfig(
    vocab_size=50257,
    block_size=128,
    n_layer=6,
    n_head=6,
    n_embd=384,
    dropout=0.1,
    bias=True
)

model = GPT(config)

In [ ]:
def estimate_loss(model):
  out = {}
  model.eval()
  with torch.inference_mode():
    for split in ['train','val']:
      losses = torch.zeros(eval_iters)
      for k in range(eval_iters):
        X,y = get_batch(split)
        with ctx:
          logits ,loss = model(X,y)
        losses[k] = loss.item()
      out[split] = losses.mean()
  model.train()
  return out

In [ ]:
import torch
from contextlib import nullcontext


learning_rate = 1e-4
max_iters = 20000
warmup_steps = 1000
min_lr = 5e-4
eval_iters = 500
batch_size = 32
block_size = 128

gradient_accumulation_steps = 32

device = "cuda" if torch.cuda.is_available() else "cpu"
device_type = "cuda" if "cuda" in device else "cpu"

dtype = "bfloat16" if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else "float16"
ptdtype = {"float32": torch.float32, "bfloat16": torch.bfloat16 , "float16": torch.float16 }[dtype]


ctx = nullcontext() if device_type == "cpu" else torch.amp.autocast(device_type = device_type,dtype = dtype)

torch.set_default_device(device)
torch.manual_seed(42)

In [ ]:
from torch.optim.lr_scheduler import  LinearLR , SequentialLR, CosineAnnealingLR

optimizer = torch.optim.AdamW(model.parameters(),lr = learning_rate,betas = (0.9,0.95),weight_decay = 0.1 , eps = 1e-9)

scheduler_warmup = LinearLR(optimizer,total_iters = warmup_steps)
scheduler_decay = CosineAnnealingLR(optimizer,max_iters - warmup_steps,eta_min = min_lr)
scheduler = SequentialLR(optimizer,schedulers = [scheduler_warmup,scheduler_decay],milestones = [warmup_steps])

scaler = torch.cuda.amp.GradScaler(enabled = (dtype == "float16"))

In [ ]:
best_val_loss = float('inf')
best_model_params_path = "best_model_params.pt"
train_loss_list , validation_loss_list = [] , []

model = model.to(device)

for epoch in tqdm(range(max_iters)):
  if epoch % eval_iters ==0 and epoch!=0:
    losses = estimate_loss(model)
    print(f"Epoch {epoch}: train_loss {losses['train']:.4f} ,val_loss {losses['val']:.4f}")
    print(f"The Current Learning Rate : {optimizer.param_groups[0]['lr']:.5f}")
    train_loss_list += [losses['train']]
    validation_loss_list += [losses['val']]

    if losses['val']< best_val_loss:
      best_val_loss = losses['val']
      torch.save(model.state_dict(),best_model_params_path)
  X,y = get_batch('train')
  X,y = X.to(device) , y.to(device)

  with ctx:
    logits , loss = model(X,y)
    loss = loss/ gradient_accumulation_steps
    scaler.scale(loss).backward()

  if ((epoch+1) % gradient_accumulation_steps ==0) or (epoch + 1 == max_iters):
    torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm = 0.5)
    scaler.step(optimizer)
    scaler.update()
    optimizer.zero_grad(set_to_none = True)
  scheduler.step()


In [ ]:
import matplotlib.pyplot as plt
train_loss_list_converted = [i.cpu().detach() for i in train_loss_list]
validation_loss_list_converted = [i.cpu().detach() for i  in validation_loss_list]

plt.plot(train_loss_list_converted,'g',label = "Train Loss")
plt.plot(validation_loss_list_converted,'r',label = "Validation Loss")
plt.xlabel("Steps - Every 100 Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
model = GPT(config)
device = "cuda" if torch.cuda.is_available() else "cpu"
best_model_params_path = "best_model_params.pt"
model.load_state_dict(torch.load(best_model_params_path,map_location=torch.device(device)))

In [ ]:
sentence = "Once upon a time there was a apple"
context = (torch.tensor(enc.encode_ordinary(sentence)).unsqueeze(dim=0))
y = model.generate(context,200)
print(enc.decode(y[0].tolist()))

In [ ]:
sentence = "Let's hope for a change"
context = (torch.tensor(enc.encode_ordinary(sentence)).unsqueeze(dim=0))
y = model.generate(context,200)
print(enc.decode(y[0].tolist()))